# Generate tsunami plots and data #

## Set up Jupyter Notebook ##

In [ ]:
#set up environment and imported modules
import os
os.environ['CLAW'] = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack'
os.environ['FC'] = 'gfortran'
dir = os.path.join(os.environ['CLAW'], 'geoclaw/examples/hidaka/scratch')

%pylab inline

from clawpack.clawutil import nbtools
from clawpack.visclaw import animation_tools
from IPython.display import HTML, Image
from clawpack.geoclaw import dtopotools
import matplotlib.pyplot as plt
plt.rcParams["animation.embed_limit"] = 500 # 20 mb default is too small

import numpy as np

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [39]:
def show_anim(anim):
    html_version = HTML(anim.to_jshtml())
    # html_version = HTML(anim.to_html5_video())
    return html_version

## Compile and run Geoclaw Code

In [13]:
nbtools.make_exe(new=True, verbose=False)  # compile xgeoclaw

In [ ]:
# setrun data
# create *.data files from parameters in setrun.py
from setrun import setrun 
rundata = setrun()
rundata.write()  

In [41]:
run maketopo.py  # download the topo file and create the dtopo file

*** Note: since grid registration is llcorner,
    will shift x,y values by (dx/2, dy/2) to cell centers
The extent of the data in longitude and latitude: 
[np.float64(135.0020833333335), np.float64(151.9979166680265), np.float64(34.5020833333335), np.float64(46.9979166676665)]
Using Okada model to create dtopo file


## Output visuals for the dtopo created in maketopo.py

In [ ]:
# copied from maketopo.py and CSZ_example.ipynb

if 1:
    fault_geometry_file = os.path.join(dir, 'urakawa_1982/fault_model_1982.csv') # finite fault mesh
    rupture_file = os.path.join(dir, 'urakawa_1982/rupt_param_1982.csv') # rupture information as [slip, rise, rupt time]

    fault_mesh = np.loadtxt(fault_geometry_file, delimiter=",", skiprows=1) #path, comma separated values, first row is a header
    fault_mesh[:,[2,5,8]] = 1e3*abs(fault_mesh[:,[2,5,8]]) #array slicing accesses depth element, changing it to be positive m instead of negative km
    rupture_parameters = np.loadtxt(rupture_file, delimiter=",", skiprows=1)

    # create fault object
    fault0 = dtopotools.Fault() #create object
    fault0.subfaults = [] #initialize empty list
    nsubfaults = fault_mesh.shape[0]

    for j in range(nsubfaults):
        subfault0 = dtopotools.SubFault() #create new object
        node1 = fault_mesh[j,0:3].tolist() #lon,lat,depth of the first node in each triangle
        node2 = fault_mesh[j,3:6].tolist()
        node3 = fault_mesh[j,6:9].tolist()
        node_list = [node1,node2,node3]
        subfault0.slip = rupture_parameters[j,1] #all dip slip
        subfault0.rake = fault_mesh[j,11]
        subfault0.set_corners(node_list, projection_zone='10')
        fault0.subfaults.append(subfault0)

    # specify extent of seafloor deformation (?)
    xlower = 140
    xupper = 145
    ylower = 41
    yupper = 44

    # dtopo parameters:
    points_per_degree = 60  # 1 minute resolution
    dx = 1./points_per_degree
    mx = int((xupper - xlower)/dx + 1)
    xupper = xlower + (mx-1)*dx
    my = int((yupper - ylower)/dx + 1)
    yupper = ylower + (my-1)*dx

    x = np.linspace(xlower,xupper,mx)
    y = np.linspace(ylower,yupper,my)

    dtopo = fault0.create_dtopography(x,y,times=[1.], verbose=False)
    print('Created %s, with dynamic rupture of a Mw %.2f event' % ("urakawa-oki", fault0.Mw()))


    # copied directly from CSZ_example, plots the fault

    fig = plt.figure(figsize=(15,10))
    #ax = fig.add_subplot(121, projection='3d')
    ax = fig.add_axes([.05,.05,.65,.9], projection='3d')
    for s in fault0.subfaults:
        c = s.corners #list of 3 nodes
        c.append(c[0]) #does this close a loop or something?
        c = np.array(c)
        ax.plot(c[:,0],c[:,1],-c[:,2]/1000.,color='b') #depth is negative and km (m/1000)
    ax.view_init(10,60)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_zlabel('Depth (km)')
    ax.set_title('Triangular subfaults')

    #ax = fig.add_subplot(122)
    ax = fig.add_axes([.75,.05,.2,.9])
    for s in fault0.subfaults:
        c = s.corners
        c.append(c[0])
        c = np.array(c)
        ax.plot(c[:,0],c[:,1], 'b')
    ax.set_aspect(1./np.cos(45*np.pi/180.))
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Plan view')


In [ ]:
# show uplift (copied from CSZ_example.ipynb)

if 1:
   plt.close('all')

   fig,(ax0,ax1) = plt.subplots(ncols=2,nrows=1,figsize=(16,6))
   fault0.plot_subfaults(axes=ax0,slip_color=True,plot_box=False);
   ax0.set_title('Slip on Fault');

   X = dtopo.X; Y = dtopo.Y; dZ_at_t = dtopo.dZ_at_t
   dz_max = dtopo.dZ.max()
   print(dz_max)


      # time to plot deformation
   dtopotools.plot_dZ_colors(X,Y,dZ_at_t(1.),axes=ax1, 
                           cmax_dZ = dz_max, add_colorbar=True);
   fault0.plot_subfaults(axes=ax1, slip_color=False, plot_box=True)
   ax1.set_title('Seafloor at time t=1');

In [ ]:
# #Animate the rupture (from CSZ_example)

# dz_max = abs(dtopo0.dZ).max()

# # Incorporate this function in dtopotools to replace animate_dz_colors?
# def plot_subfaults_dZ(t, fig):
#     fig.clf()
#     ax1 = fig.add_subplot(121)
#     ax2 = fig.add_subplot(122)
#     fault0.plot_subfaults(axes=ax1, slip_color=True, plot_box=False,
#                           slip_time=t)
#     dtopo0.plot_dZ_colors(axes=ax2, t=t, cmax_dZ=dz_max)
#     return fig

# figs = []
# times = dtopo0.times[::5]   # only use every 5th time for animation
# if dtopo0.times[-1] not in times:
#     times = np.hstack((times, dtopo0.times[-1]))  # include final dz
    
# print('Animation will include %i times' % len(times))
# for k,t in enumerate(times):
#     fig = plt.figure(figsize=(12,5))
#     plot_subfaults_dZ(t,fig)
#     figs.append(fig)
#     plt.close(fig)

# anim = animation_tools.animate_figs(figs, figsize=(12,6))
# HTML(anim.to_jshtml())

## Generate tsunami animation and plots

In [ ]:
# function definitions for 'make' subprocesses
# function copied and edited from the one given in nbtools.py that wouldn't work right

def make_driver(args, env, outfile, verbose):

    """
    Use subprocess to run a make command and capture the output.

    :Input:
    - *args*: arguments to the make command
    - *env*: environment to run in (if None, use *os.environ*)
      Useful if $CLAW must be set in notebook.
    - *outfile*: file name of file for capturing stdout and stderr
    - *verbose*: if True, print out command and display link to output file
    
    If the return code is non-zero, print a warning and link to output file
    regardless of value of *verbose*.

    """

    import subprocess # execute terminal commands in python
    import os, sys

    if env is None:
        env = os.environ

    if verbose:
        print("Executing shell command:   make %s" % args)
        sys.stdout.flush()

    ofile = open(outfile,'w')
    cmd_list = ['make'] + args.split()

    # location of Makefile
    directory = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido' 
    cmd_list.insert(1, "-C")
    cmd_list.insert(2, directory)

    job = subprocess.Popen(cmd_list, stdout=ofile,stderr=ofile,env=env)
    return_code = job.wait()
    errors = (return_code != 0)
    if errors:
        print("*** Possible errors, return_code = %s" % return_code)
    
    if verbose or errors:
        local_file = FileLink(outfile)
        print("Done...  Check this file to see output:") 
        display(local_file)

def make_plots(label=None, env=None, verbose=True):
    """Perform 'make plots' and display links"""

    if label is None: 
        label = ''
    else:
        if label[0] != '_':
            label = '_' + label
    #outdir = '_output%s' % str(label)
    outdir = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido/_output'
    #plotdir = '_plots%s' % str(label)
    plotdir = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido/_plots'
    outfile = 'plot_output%s.txt' % str(label)

    args = '.plots OUTDIR=%s PLOTDIR=%s' % (outdir,plotdir)
    #args = '.plots'
    make_driver(args, env, outfile, verbose)

    if verbose:
        index_file = FileLink('%s/_PlotIndex.html' % plotdir)
        print("View plots created at this link:")
        display(index_file)

    return plotdir

In [ ]:
# run make .plots (LONG RUNTIME)

plotdir = make_plots(verbose=False)

In [ ]:
#plotdir = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido/_plots'
anim = animation_tools.animate_from_plotdir(plotdir, figno=0) #figno=1 shows cropped area around hokkaido coastline
show_anim(anim) 

## output gauge data

In [ ]:
# Gauge located near Urakawa 142.755637, 42.158995
# the earthquake happened at 11:32 JST, time for csv files begins around 11 JST
from setplot import setplot
gauge_path = os.path.join(dir, 'urakawa_raw_observations.csv') # observed tide gauge data
tide_path = os.path.join(dir, 'urakawa_tidal_component.csv')
raw_observations = np.loadtxt(gauge_path, delimiter=',', skiprows=1) # x,y time,height
tidal_component = np.loadtxt(tide_path, delimiter=',', skiprows=1)

tide_data = np.empty((raw_observations.shape[0], 4))

tide_data[:,0] = raw_observations[:,0] # time JST
tide_data[:,1] = raw_observations[:,1] # tide gauge measurement
tidal_component = np.vstack((tidal_component[0], tidal_component)) # one data point short by accident
tide_data[:,2] = tidal_component[:,1] # don't need times, only data
tide_data[:,3] = tide_data[:,1] - tide_data[:,2]

# grab the correct times, data begins around 10:00 JST, but we only need 11:30 and beyond
tide_data[:,0] = tide_data[:,0]*60 # convert to minutes
event_time = 11.5 * 60
sorting = tide_data[:,0] > event_time

gauge_data = tide_data[::][sorting]
gauge_data[:,0] = gauge_data[:,0] - event_time # time shift so that time 0 is the earthquake

plotdata = setplot()
time_shift = 10 # 10 minutes
plotdata.outdir = '_output'
g129 = plotdata.getgauge(129)
t = (g129.t / 60.) + time_shift # convert to minutes 
eta = g129.q[3,:]   # eta = h + B (depth plus bathymetry)
plot(t,eta, 'r.-', markersize=1, label='GeoClaw')
plot(gauge_data[:,0], gauge_data[:,3]/100, 'k.-', markersize=5, label='Observation') # convert cm to m
plot(t,0*t, 'k-', label='Sea level', linewidth=0.5)
xlim(0,240)
ylim(-1.5, 1.5)
xlabel('minutes since earthquake')
ylabel('meters')
title('Sea surface elevation at Urakawa')
grid(True)
legend()

In [ ]:
# Gauge located near Erimo
from setplot import setplot
gauge_path = os.path.join(dir, 'erimo_tidal_observations.csv') # observed tide gauge data
observed_erimo = np.loadtxt(gauge_path, delimiter=',', skiprows=1) # x,y time,height

plotdata = setplot()
plotdata.outdir = '_output'
g112 = plotdata.getgauge(112)
#t = (g111.t / 3600.) # convert to hours 
t = (g112.t / 60.) # convert to minutes 
eta = g112.q[3,:]   # eta = h + B (depth plus bathymetry)
plot(t,eta, 'r.-', markersize=1, label='GeoClaw')
plot(observed_erimo[:,0]*60 + 10, (observed_erimo[:,1]/100)-0.04, 'k.-', markersize=5, label='Observation') # cm to m in erimo observations
plot(t,0*t, 'k-', label='Sea level', linewidth=0.5)
xlim(0,120)
ylim(-1.5, 1.5)
xlabel('Minutes since earthquake')
ylabel('meters')
title('Sea surface elevation at Erimo')
grid(True)
legend()

In [ ]:
# Gauge located near Tomakomai
# time of file begins at earthquake time, and is in hours
from setplot import setplot
gauge_path = os.path.join(dir, 'tomakomai_raw_observations.csv') # observed tide gauge data
tide_path = os.path.join(dir, 'tomakomai_tidal_component.csv')
observed_tomakomai = np.loadtxt(gauge_path, delimiter=',', skiprows=1) # x,y time,height
tomakomai_tide = np.loadtxt(tide_path, delimiter=',', skiprows=1)

plotdata = setplot()
plotdata.outdir = '_output'
g113 = plotdata.getgauge(113)
#t = (g111.t / 3600.) # convert to hours 
t = (g113.t / 60.) + time_shift # convert to minutes and shift
eta = g113.q[3,:]   # eta = h + B (depth plus bathymetry)
plot(t,eta, 'r.-', markersize=1, label='GeoClaw')
plot(observed_tomakomai[:,0]*60, (observed_tomakomai[:,1]-tomakomai_tide[:,1])/100, 'k.-', markersize=5, label='Observation') # hours to min conversion in time
plot(t,0*t, 'k-', label='Sea level', linewidth=0.5)
xlim(0,120)
ylim(-0.5, 0.5)
xlabel('Minutes since earthquake')
ylabel('meters')
title('Sea surface elevation at Tomakomai (+ timeshift)')
grid(True)
legend()

## FGMax read and plot

In [ ]:
# read in fgmax data

# for rundata.fgmax_data.num_fgmax_val = 1 : 
# columns in file: lon, lat, amr_level, topo value B, maximum depth h, time of max h, arrival time 

from clawpack.visclaw import colormaps
from clawpack.visclaw import plottools
from clawpack.visclaw.plottools import pcolorcells
from clawpack.geoclaw import fgmax_tools
from matplotlib import colors 
import os,sys
import glob

fg = fgmax_tools.FGmaxGrid()
outdir = '_output'
fg.outdir = outdir
data_file = os.path.join(outdir, 'fgmax_grids.data')
fg.read_fgmax_grids_data(fgno=1, data_file=data_file) # currently only one fgmax grid used
fg.read_output()

In [ ]:
# plot initial and end topography

# In the plot above, "GeoClaw B" refers to the cell-averaged topography value used by GeoClaw 
# and stored with the fgmax output, and is generally recorded after any subsidence/uplift. 
#  The colors are blues for values of $B < 0$ and greens/brown for $B > 0$.  

zmin = -60.
zmax = 20.
land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})

sea_cmap = colormaps.make_colormap({ 0.0:[0,0,1], 1.:[.8,.8,1]})

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=(zmin,zmax),
                                     data_break=0.)                                   

fname = os.path.join(dir, 'Hokkaido_B0.txt')
B0 = np.loadtxt(fname)
B0_masked = np.ma.masked_array(B0, fg.B.mask)
fg.dz = fg.B - B0_masked
fg.B0 = B0_masked

plt.close('all')
fig, ax = plt.subplots()
figsize=(8,8)
pc = pcolorcells(fg.X, fg.Y, fg.B, cmap=cmap, norm=norm)  
cb = plt.colorbar(pc,shrink=0.5,extend='both')
cb.set_label('meters')
cb.set_ticks(np.hstack((np.linspace(zmin,0,5), np.linspace(0,zmax,5))))
ax.set_aspect(1./cos(48*pi/180.))
ax.ticklabel_format(useOffset=False)
plt.xticks(rotation=20)
ax.set_title('GeoClaw B topography on fg1 grid')
plt.show()

fig, ax = plt.subplots()
figsize=(8,8)
pc = pcolorcells(fg.X, fg.Y, fg.B0, cmap=cmap, norm=norm)  
cb = plt.colorbar(pc,shrink=0.5,extend='both')
cb.set_label('meters')
cb.set_ticks(np.hstack((np.linspace(zmin,0,5), np.linspace(0,zmax,5))))
ax.set_aspect(1./cos(48*pi/180.))
ax.ticklabel_format(useOffset=False)
plt.xticks(rotation=20)
ax.set_title('GeoClaw B0 topography on fg1 grid')
plt.show()

In [ ]:
# plot onshore inundation
# code taken from https://depts.washington.edu/ptha/WA_EMD_2021/ process_fgmax.py
# For plotting the inundation depth on shore without showing offshore depths, and for plotting `eta = B + h` only offshore.
# read in timing data

t_files = glob.glob(outdir + '/fort.t0*') # grabs all the timing files 
times = []
for f in t_files:
    lines = open(f,'r').readlines()
    for line in lines:
        if 'time' in line: 
            t = float(line.split()[0])
    times.append(t)
times.sort()
print('Output times found: ',times)
if len(times) > 0:
    t_hours = times[-1] / 3600.
    print('\nfgmax results are presumably from final time: %.1f seconds = %.2f hours'\
          % (times[-1], t_hours))
else:
    t_hours = np.Nan


# read in initial topography/bathymetry values for fgmax grid before dtopo event
fname = os.path.join(dir, 'Hokkaido_B0.txt')
print('Loading B0 from %s' % fname)
B0 = np.loadtxt(fname)
B0_masked = np.ma.masked_array(B0, fg.B.mask)
fg.dz = fg.B - B0_masked
fg.B0 = B0_masked

print('Maximum dz between fg.B and new fg.B0 = %.3f'  % abs(fg.B0 - fg.B).max())

# find onshore and offshore points of the grid
onshore = fg.B0 >  0.
if fg.force_dry_init is not None:
    onshore = np.logical_or(onshore, fg.force_dry_init)
offshore = np.logical_not(onshore)

fg.h_onshore = np.ma.masked_where(offshore, fg.h)
fg.eta_offshore = np.ma.masked_where(onshore, fg.B0 + fg.h)  # use B0 for continuity at shore

# ## Plot maximum flow depth
bounds_depth = np.array([1e-6,0.5,1.0,1.5,2,4.0,6.0]) 
cmap_depth = colors.ListedColormap([[.7,.7,1],[.5,.5,1],[0,0,1], [1,.7,.7], [1,.4,.4], [1,0,0]])

# Set color for value exceeding top of range to purple:
cmap_depth.set_over(color=[1,0,1])

# Set color for land points without inundation to light green:
cmap_depth.set_under(color=[.7,1,.7])
norm_depth = colors.BoundaryNorm(bounds_depth, cmap_depth.N)
    
plt.close('all')
fig, ax = plt.subplots()
figsize=(8,10)
pc = plottools.pcolorcells(fg.X, fg.Y, fg.h_onshore, cmap=cmap_depth, norm=norm_depth)
cb = plt.colorbar(pc, extend='max', shrink=0.7)
cb.set_label('meters')
plt.contour(fg.X, fg.Y, fg.B0, [0], colors='g')

ax.set_aspect(1./np.cos(48*np.pi/180.))
plt.ticklabel_format(useOffset=False)
plt.xticks(rotation=20)
plt.title('Maximum flow depth over %.2f hours' % t_hours)
plt.show()

# In the plot above, green shows fgmax points that never got wet.  
# The green contour shows `B0 = 0`, and note that some of the initially dry region below MHW never got wet 
# (over the limited duration of this simulation).
# 
# White areas are masked out either because they were not fgmax points or because they were initially wet. 
# 
# Regions colored blue or red are initially dry fgmax points that did get wet during the tsunami, 
# with color showing the maximum depth of water recorded.

In [ ]:
# maximum amplitude and timing plots (I have no idea where this is from?)
from clawpack.visclaw import geoplot


outdir = '_output'
plotdir = '_plots'

fg = fgmax_tools.FGmaxGrid()
fg.outdir = outdir
data_file = os.path.join(outdir, 'fgmax_grids.data')
fg.read_fgmax_grids_data(fgno=1, data_file=data_file)
fg.read_output()

# color bar lines
clines_zeta = [0.01] + list(np.linspace(0.05,0.3,6)) + [0.5,1.0,10.0]
colors = geoplot.discrete_cmap_1(clines_zeta)
plt.figure(1)
plt.clf()

zeta = np.where(fg.B>0, fg.h, fg.h+fg.B)   # surface elevation in ocean
plt.contourf(fg.X,fg.Y,zeta,clines_zeta,colors=colors)
plt.colorbar()
plt.contour(fg.X,fg.Y,fg.B,[0.],colors='k')  # coastline (where height is zero?)

# plot arrival time contours and label:
arrival_t = fg.arrival_time/3600.  # arrival time in hours
clines_t = np.linspace(0,1,10)  # hours
clines_t_label = clines_t[2::2]  # which ones to label 
clines_t_colors = ([.5,.5,.5],)
con_t = plt.contour(fg.X,fg.Y,arrival_t, clines_t,colors=clines_t_colors) 
plt.clabel(con_t, clines_t_label)

# fix axes:
plt.ticklabel_format(style='plain',useOffset=False)
plt.xticks(rotation=20)
plt.gca().set_aspect(1./np.cos(fg.Y.mean()*np.pi/180.))
plt.title("Maximum amplitude / arrival times")
plt.show()